In [ ]:
!pip install alpaca-trade-api
!pip install transformers
!pip install nltk
!pip install ipython ipykernel --upgrade
# !pip install pandas-profiling

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached ipykernel-6.15.1-py3-none-any.whl (132 kB)
  Attempting uninstall: ipykernel
    Found existing installation: ipykernel 5.3.4
    Uninstalling ipykernel-5.3.4:


In [ ]:
import alpaca_trade_api as alp
from transformers import pipeline
import keras
import tensorflow as tf
import matplotlib.pyplot as plt
import sys, os, glob, random, time, datetime
import numpy as np
import sklearn as sk
import pandas as pd
import re
import nltk
from collections import deque

Retrieve market data 


In [ ]:
from alpaca_trade_api.rest import REST, TimeFrame, TimeFrameUnit
# Secret key must to be changed if regenerating the key id
api_key = 'AK2BNNAGRJPM2CZBC488'
secret_key = 'Gi5WCKBIxf5CkTSBor5jiFS910n8TqAbbKQzUT4V'

# api = REST(api_key, secret_key)
# start_date = '2021-01-01'; end_date = '2022-01-01'
# btc_bars = api.get_crypto_bars('BTCUSD', TimeFrame.Day, start_date, end_date).df # get_trades, get_quotes, ETHUSD
# btc_bars
# stock_choice = 'TSLA' # 'AAPL','AMD','AMZN','FB','GOOG','GOOGL','MSFT','NFLX','NVDA'
# stock_bars = api.get_bars(stock_choice, TimeFrame.Day, start_date, end_date).df
# stock_bars

Retrieve news data

In [ ]:
from alpaca_trade_api.stream import Stream
# For historic data use the REST class, for live data use the Stream class
rest_client = REST(api_key, secret_key)
stream_client = Stream(api_key, secret_key)

Transformer sentiment analysis using (HuggingFace) BERT network

In [ ]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')
model_names = ['distilbert-base-uncased-finetuned-sst-2-english',
               'finiteautomata/bertweet-base-sentiment-analysis',
               'fhamborg/roberta-targeted-sentiment-classification-newsarticles',
               'mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis']

classifier = pipeline('sentiment-analysis', model = model_names[-1])

def news_data_processor(news):
  date = str(news.created_at)[:10]
  summary = news.summary
  headline = news.headline
  text = headline + ' ' + summary
  word_list = word_tokenize(text)
  words = [word for word in word_list if word.isalpha()]

  stop_words = set(stopwords.words('english'))
  filtered_words = [w for w in words if not w in stop_words]

  sentence = ' '.join(filtered_words)
  sentiment = classifier(sentence)

  return sentence, sentiment, date

In [ ]:
class DQRL_Trader():

  def __init__(self, state_size, action_space=3):
    self.state_size = state_size
    self.action_space = action_space    
    self.memory = deque(maxlen=1500)
    self.inventory = []
    self.n_data_inputs = 6 # open, open_avg_diff_5_days, open_avg_diff_5_days, portfolio_value, number_of_stocks, sentiment (=6 if including sentiment)

    self.gamma = 0.95
    self.epsilon = 1.0
    self.epsilon_min = 0.1
    self.epsilon_decay = 0.995
    self.learning_rate = 0.001

    self.model = self.model_builder()

  def model_builder(self):
    init = tf.keras.initializers.HeUniform(seed=1)
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(64, activation='relu', input_shape=(self.n_data_inputs,1), kernel_initializer=init),
        tf.keras.layers.Dense(64, activation='relu', kernel_initializer=init),
        tf.keras.layers.Dense(64, activation='relu', kernel_initializer=init),
        # tf.keras.layers.Dense(64, activation='relu', kernel_initializer=init),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(self.action_space, activation='linear', kernel_initializer=init)
    ])
    model.compile(loss=tf.keras.losses.Huber(), optimizer=tf.keras.optimizers.Adam(learning_rate=self.learning_rate))
    return model 

  def trade(self, state):
    if random.random() <= self.epsilon:
      return random.randrange(self.action_space)
    else:
      options = self.model.predict(state)
      # print('options', options)
    return np.argmax(options[0])

  def batch_Q_learn(self, batch_size):
    mini_batch = []
    Mem = len(self.memory)
    # print('len(self.memory) = ', Mem)
    for i in range(Mem - batch_size + 1, Mem):
      mini_batch.append(self.memory[i])

    for state, action, reward, next_state, done in mini_batch:
      target = reward
      if not done:
        target = reward + self.gamma * np.amax(self.model.predict(next_state))

      target_func = self.model.predict(state)
      # print('state ', state)
      # print('action ', action)
      target_func[0][action] = target
      self.model.fit(state, target_func, epochs=1, verbose=0)

      if self.epsilon > self.epsilon_min:
        self.epsilon *= self.epsilon_decay


In [ ]:
def price_format(p):
  return ('-$' if p<0 else '$') + '{0:.2f}'.format(abs(p))

def sigmoid(x):
  return 1/(1+np.exp(-x))

In [ ]:
# N-day state representation ending at time t
def create_state(data, time_step, window_size, portfolio, n_stocks):
  start_index = time_step - window_size + 1
  data_window = data[start_index : time_step+1] if start_index >= 0 else (-start_index * [data[0]]) + list(data[0 : time_step+1])
  # print(len(data_window), window_size)
  state=[]
  for i in range(len(data_window) -1):
    indicators = sigmoid(data_window[i+1] - data_window[i]).flatten()
    joint = [*indicators, portfolio, n_stocks]
    state.append(joint)
    # print(sigmoid(data_window[i+1] - data_window[i]))

  return np.array(state).astype('float32')

In [ ]:
start_date = '2020-01-01'; end_date = '2022-01-01'
stock_choice = 'TSLA' # 'AAPL','AMD','AMZN','FB','GOOG','GOOGL','MSFT','NFLX','NVDA'
stock_bars = rest_client.get_bars(stock_choice, TimeFrame.Day, start_date, end_date).df

# stream_client.subscribe_news(live_news_data, stock_choice)
# stream_client.run()

def create_date_arr(data):
  stock_date_list = []
  for i in range(len(data)):
    date_str = str(data.index[i])[:10]
    stock_date_list.append(date_str)
  return np.array(stock_date_list)

stock_open = stock_bars['open']
stock_dates = create_date_arr(stock_bars)
# print(stock_dates)

# Compute the difference between the opening price of a day with the mean opening price over a previous set of days e.g. 5 and 50
def avg_open_diff(open_price_data, n_days, start_index):
  # if start_index < n_days:
  #   print('Beware the moving average difference in opening price is only computed over the last {} day(s), as opposed to {}.'.format(start_index, n_days))
  if start_index < len(open_price_data):
    start_price = open_price_data[start_index]
    prior_avg_price = open_price_data[start_index-n_days : start_index].mean()
    diff = start_price - prior_avg_price if start_index-n_days >= 0 else start_price - open_price_data[:start_index].mean()
    return diff
  else:
    return IndexError('Failed to compute moving average difference: invalid request')

def create_avg_diff_data(data, n):
  stock_avg_diff_n = [0] # avg diff = 0 for first open price
  for i in range(1, len(data)):
    stock_avg_diff_n_i = avg_open_diff(data, n, i)
    stock_avg_diff_n.append(stock_avg_diff_n_i)
  return np.array(stock_avg_diff_n)

stock_avg_diff_5 = create_avg_diff_data(stock_open, 5)
stock_avg_diff_50 = create_avg_diff_data(stock_open, 50)

stock_data = np.transpose([stock_open, stock_avg_diff_5, stock_avg_diff_50])#.astype(np.float16)

news = rest_client.get_news(stock_choice, start_date, end_date, limit=10000)

N_news = len(news)
NEG, POS, NEU = 0, 0, 0
news_data_arr = []
news_dates = []
for i in range(N_news):
  article_text, sentim, news_date = news_data_processor(news[i])
  if news_date in stock_dates:
    # print(news_date)
    sentim_label = sentim[0]['label']
    sentim_score = float(sentim[0]['score'])
    if sentim_label == 'negative':
      score = sentim_score * -1
      # NEG+=1
    elif sentim_label == 'positive':
      score = sentim_score
      # POS+=1
    elif sentim_label == 'neutral':
      score = 0.0
      # NEU+=1
    news_data_arr.append([score, news_date])
    news_dates.append(news_date)
    # news_data_arr.insert(i, [article_text, sentim[0]['label'], sentim[0]['score'], news_date]) if reversing time ordering (as news data is read in from end -> start date)
    # news_data_arr.append([article_text, sentim[0]['label'], sentim[0]['score'], news_date])
  else:
    continue

# print(NEG, POS, NEU)
# print(news_data_arr)
news_data_reverse = news_data_arr[::-1]
news_dates_reverse_array = np.asarray(news_data_reverse)

# Sort by date-time
def sort_DT(data):
  return sorted(data, key=lambda x: datetime.datetime.strptime(x, '%Y-%m-%d'))

unique_sorted_news_dates = np.array(sort_DT(list(set(news_dates))))
sentiment_scores = []
for unique_date in unique_sorted_news_dates:
  # print(unique_date)
  date_indexes = np.where(unique_date == news_dates_reverse_array)[0]
  # print(news_dates_reverse_array[index_,0])
  float_scores = [float(i) for i in news_dates_reverse_array[date_indexes,0]]
  avg_score = np.mean(float_scores)
  sentiment_scores.append(avg_score)

sentiment_scores = np.array(sentiment_scores)

if len(stock_data) != len(sentiment_scores):
  print('Error in processing sentiment scores/ start-end dates misaligned for news and market data')


In [ ]:
stock_sentiment_data = np.hstack((stock_data, np.expand_dims(sentiment_scores, axis=1)))
print(stock_sentiment_data)

In [ ]:
window_size = 5 # days inbetween action e.g. buy/sell/sit
epochs = 1000
batch_size = 32
data = stock_sentiment_data     # stock_data     
N_data = len(data)-1

trader = DQRL_Trader(window_size)
trader.model.summary()

In [ ]:
for epoch in range(epochs):
  print('Epoch {}/{} ...'.format(epoch+1, epochs))

  # state = create_state(data, 0, window_size+1)
  trader.inventory = []
  total_profit = 0
  n_stocks_owned = 10
  initial_funds = 1000
  portfolio_value = (n_stocks_owned * data[0][0]) + initial_funds
  print('Initial portfolio value: {} | Number of stocks: {}'.format(price_format(portfolio_value), n_stocks_owned))

  state = create_state(data, 0, window_size+1, portfolio_value, n_stocks_owned)
  # print(state.shape)

  for t in range(N_data):
    action = trader.trade(state)
    # next_state = create_state(data, t+1, window_size+1)
    next_state = create_state(data, t+1, window_size+1, portfolio_value, n_stocks_owned)
    # print(next_state.shape)
    reward = 0

    if portfolio_value < data[t][0] or n_stocks_owned == 0: # punish inability to buy/sell stock
      reward = -10

    if portfolio_value < data[t][0] and n_stocks_owned == 0: # huge penalty for bankcruptcy
      reward = -100 

    if action == 0: # punish repeated sitting over time
      print('Sitting...')
      reward = -0.1
    
    elif action == 1 and portfolio_value >= data[t][0]: # buy
      n_stocks_owned += 1
      portfolio_value -= data[t][0]
      trader.inventory.append(data[t])
      print('Trader bought a stock for {}'.format(price_format(data[t][0])))
      print('Total portfolio value: {} | Number of stocks: {}'.format(price_format(portfolio_value), n_stocks_owned))

    elif action == 2 and len(trader.inventory) > 0: # sell
      bought_price = trader.inventory.pop(0)
      n_stocks_owned -= 1
      portfolio_value += data[t][0]
      # print('bought_price: ', bought_price)
      if data[t][0] - bought_price[0] > 0:
        reward = 1
      elif data[t][0] - bought_price[0] == 0:
        reward = 0
      else:
        reward = -1

      total_profit += data[t][0] - bought_price[0]
      print('Trader sold one stock at {} | Transaction profit: {}'.format(price_format(data[t][0]), price_format(data[t][0] - bought_price[0])))
      print('Total portfolio value: {} | Number of stocks: {}'.format(price_format(portfolio_value), n_stocks_owned))

    done = True if t == N_data-1 else False

    trader.memory.append([state, action, reward, next_state, done])
    # print('memory', trader.memory)
    state = next_state

    if done:
      print('\n Total profit = {} \n'.format(price_format(total_profit)))

    if len(trader.memory) > batch_size:
      trader.batch_Q_learn(batch_size)
    else:
      continue

In [ ]:
# state_data, action_data = [], []
# for i in range(len(trader.memory)):
#   state_data.append(np.squeeze(trader.memory[i][0]))
#   action_data.append(np.squeeze(trader.memory[i][1]))  
